## 데이터 전처리

In [1]:
import torch

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

print(device)

cuda


In [3]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

In [4]:
data_transforms = {
    'train' : transforms.Compose([
        transforms.Resize([64, 64]),
        transforms.RandomHorizontalFlip(),# 랜덤으로 수평으로 뒤집음
        transforms.RandomVerticalFlip(), # 랜덤으로 수직으로 뒤집음
        transforms.RandomCrop(52), # 이미지 랜덤으로 Crop(임의의 지점을 중심으로 확대)
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.485, 0.406],
                            [0.229, 0.224, 0.225])
    ]),
    'val' : transforms.Compose([
        transforms.Resize([64, 64]),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.485, 0.406],
                            [0.229, 0.224, 0.225])
    ])
}

In [5]:
import os
import shutil

In [6]:
data_dir = './splitted'
image_datasets = {x: ImageFolder(root=os.path.join(data_dir, x),
                                transform = data_transforms[x]) for x in ['train', 'val']}

In [7]:
batch_size = 128 ; epochs = 30

In [8]:
dataloders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                            batch_size = batch_size, shuffle = True, num_workers=4) for x in ['train', 'val']}

In [9]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

class_names = image_datasets['train'].classes

## 모델링(Pretrained 모델 불러오기)

In [10]:
from torchvision import models

In [11]:
resnet = models.resnet50(pretrained=True)

In [12]:
num_ftrs = resnet.fc.in_features
print(num_ftrs)

2048


마지막 출력 채널의 개수가 2048이다. 그러나 우리는 33개 이므로 이를 바꿔줘야됨.

In [13]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [14]:
resnet.fc = nn.Linear(num_ftrs, 33)
resnet = resnet.to(device)

In [15]:
criterion = nn.CrossEntropyLoss()

In [16]:
optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr=0.001)

requires_grad = True로 설정된 Layer에 대해서만 가중치 갱신이 이루어짐

In [17]:
from torch.optim import lr_scheduler

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

7 epochs 마다 0.1을 곱하여 learning_rate를 감소시킴

In [18]:
ct = 0
for child in resnet.children():
    ct += 1
    if ct < 6:
        for param in child.parameters():
            param.requires_grad = False

In [19]:
print(ct)

10


즉 resnet에는 10개의 layer가 있고, 이중 상위 5개의 layer는 param.requires_grad = False를 통해 가중치 갱신을 안함

In [20]:
import copy
import time

In [21]:
def train_resnet(model, criterion, optimizer, scheduler, num_epochs=25):
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f'------------epoch {epoch+1}------------')
        since = time.time()
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
            
            for inputs, labels in dataloders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):# phase == 'train'일때만 가중치 갱신되도록 설정
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                running_loss += loss.item() * inputs.size(0) # 모든 loss를 합산하여 미니배치 수를 곱하여 계산 
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()
                l_r = [x['lr'] for x in optimizer_ft.param_groups]
                print('learning rate:', l_r)
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
        time_elapsed = time.time() - since
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed //60, time_elapsed % 60))
        
    print('Best val Acc: {:4f}'.format(best_acc))
    
    model.load_state_dict(best_model_wts)
    
    return model

In [22]:
model_resnet = train_resnet(resnet, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)

------------epoch 1------------
learning rate: [0.001]
train Loss: 0.6309 Acc: 0.8129
val Loss: 0.3670 Acc: 0.8802
Completed in 2m 9s
------------epoch 2------------
learning rate: [0.001]
train Loss: 0.2633 Acc: 0.9149
val Loss: 0.2124 Acc: 0.9353
Completed in 2m 6s
------------epoch 3------------
learning rate: [0.001]
train Loss: 0.1903 Acc: 0.9379
val Loss: 0.1745 Acc: 0.9405
Completed in 2m 6s
------------epoch 4------------
learning rate: [0.001]
train Loss: 0.1449 Acc: 0.9520
val Loss: 0.1244 Acc: 0.9586
Completed in 2m 8s
------------epoch 5------------
learning rate: [0.001]
train Loss: 0.1265 Acc: 0.9591
val Loss: 0.1302 Acc: 0.9559
Completed in 2m 6s
------------epoch 6------------
learning rate: [0.001]
train Loss: 0.1034 Acc: 0.9666
val Loss: 0.0961 Acc: 0.9680
Completed in 2m 7s
------------epoch 7------------
learning rate: [0.0001]
train Loss: 0.1010 Acc: 0.9665
val Loss: 0.1090 Acc: 0.9650
Completed in 2m 7s
------------epoch 8------------
learning rate: [0.0001]
train

In [23]:
torch.save(model_resnet.state_dict(), 'resnet50.pt')

## 모델 평가

In [25]:
resnet = models.resnet50()
resnet.fc = nn.Linear(num_ftrs, 33)
resnet = resnet.to(device)

In [26]:
resnet.load_state_dict(torch.load('resnet50.pt'))

<All keys matched successfully>

In [27]:
resnet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [28]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            pred = output.max(1, keepdim=True)[1] # output 중 결과값 제일 큰거 출력
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    
    return test_loss, test_accuracy

In [29]:
transform_resNet = transforms.Compose([
    transforms.Resize([64, 64]),
    transforms.RandomCrop(52),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.485, 0.406],
                            [0.229, 0.224, 0.225])
])

In [30]:
test_resNet = ImageFolder(root='./splitted/test', transform=transform_resNet)

In [31]:
test_loader_resNet = torch.utils.data.DataLoader(test_resNet, batch_size=batch_size, shuffle = True, num_workers = 4)

In [32]:
test_loss, test_accuracy = evaluate(resnet, test_loader_resNet)

In [33]:
print('ResNet test acc:', test_accuracy)

ResNet test acc: 99.19889848541744
